# Analysis of FRS pseudo-assembly and library alignement by Hisat2 and STAR aligners

   In order to identify the retained introns in contigs, the strategy to identify candidates is based on split-reads when we re-aligning reads on contigs. Therefore, here, we'll compare two common aligners known to produce these split reads : it's Hisat2 (Kim et al.,2015) and STAR (Doblin et al.,2013). The two aligners will be tested on their ability to retrieve inserted introns in a Full-Random Simulated (FRS) pseudo-assembly (i.e. the length and sequence of the contigs and inserted introns but also the insertion position are totally random simulated) when the contig is only present with intron inthe assembly and when the contig is in two state in the assembly (with and without intron). 
   
### Table of contents

* [Description of FRS (Full-Random Simulation) dataset](#desc_frs)
    * [Description of the three FRS pseudo-assemblies](#desc_pa)
        * [**Table 1** *Comparison  of the three pseudo-assembly with Assemblathon.pl statistics*](#table_1)
        * [**Fig 1** *Checking of intron insertion in the pseudo-assemblies*](#fig_1)
    * [Description of the FRS reads library](#desc_lib)
        * [**Fig 2** *Checking of abundance model of the library*](#fig_2)
    * [Description of the four pseudo-assembly/library alignments performed with Hisat2 and STAR](#desc_bam)
        * [**Table 2** *Comparison  of the four alignements with samtools flagstats statistics*](#table_2)
        * [**Fig 3** *Dotplots of samtools idxstats results (contigs alignement covering) beetween the alignements of Hisat2 and Star for each reference pseudo-assembly.*](#fig_3)
* [Analysis of alignments by seaching split reads  and comparing with simulated introns](#ana_frs)
    * [Split read signal analysis](#ana_signal)
        * [**Fig 4** *Effectives table and barplots of split reads signal detection for STAR and HiSAT2 for the two types of reference.*](#fig_4)
        * [**Fig 5** *Counting table and barplots of mapped covering reads' main characteristics.*](#fig_5)
        * [**Fig 6** *Counting table and barplots of aligners' duplication capacity of covering reads in mix-states reference.*](#fig_6)

In [75]:
#Useful modules :
import re
import pickle
import os
import pandas as pd
import pysam
import gzip
from pprint import pprint
from Bio import SeqIO
from collections import OrderedDict
import plotly as py
from plotly import tools
import plotly.graph_objs as go
import plotly.subplots as psp
import plotly.io as pio
from ipywidgets import widgets
import numpy as np



py.offline.init_notebook_mode(connected=True)

if not os.path.exists("./.FRS_Alignment_validation_env/") :
    os.mkdir("./.FRS_Alignment_validation_env/")

#Useful functions :
def parsing_test(items) :
    items_of_interest = ["Number of contigs","Total size of contigs","Longest contig","Shortest contig","Number of contigs > 1K nt","N50 contig length","L50 contig count"]
    if len(items) == 2 and items[0] in items_of_interest :
        return True ;
    else :
        return False ;

def parse_assemblathon(filename : str, name : str ) :
    with open(filename,"r") as f :
        assemblathon = { re.split("\s\s+",line.strip(),1)[0] : re.split("\s\s+",line.strip(),1)[1] for line in f if parsing_test(re.split("\s\s+",line.strip(),1))}
    return pd.DataFrame(data=assemblathon.values(),index=assemblathon.keys(),columns=[name])




In [76]:
mapping_hisat_all = pd.read_pickle("./.FRS_Alignment_validation_env/reads_mapping_Hisat_-_All_with_introns")
mapping_hisat_mixed = pd.read_pickle("./.FRS_Alignment_validation_env/reads_mapping_Hisat_-_Mixed-states")
mapping_star_all = pd.read_pickle("./.FRS_Alignment_validation_env/reads_mapping_Star_-_All_with_introns")
mapping_star_mixed = pd.read_pickle("./.FRS_Alignment_validation_env/reads_mapping_Star_-_Mixed-states")
library = pd.read_pickle("./.FRS_Alignment_validation_env/FRS_aln-vld_wo-introns_library_res")
reference_mixed = pd.read_pickle("./.FRS_Alignment_validation_env/FRS_align_validation_half-wi-introns.fa.gz")
reference_all = pd.read_pickle("./.FRS_Alignment_validation_env/FRS_align_validation_all-wi-introns.fa.gz")
introns = pd.read_pickle("./.FRS_Alignment_validation_env/FRS_align_validation_introns.txt.gz")
names = ['All with introns - Hisat2','Mix-states contigs - Hisat2','All with introns - STAR','Mix-states contigs - STAR'] 
colors = {'All with introns - Hisat2':"limegreen",
          'Mix-states contigs - Hisat2':"forestgreen",
          'All with introns - STAR':"darkorange",
          'Mix-states contigs - STAR':"chocolate"}

In [77]:
mapping_star_all = mapping_star_all.merge(
            library,how='outer',
            left_on='read',
            right_index=True,
            suffixes=('','_lib')
        ).loc[
            :,
            [c if c != 'covering' else 'covering_lib' for c in mapping_star_all.columns.to_list()]
        ].rename(columns={'covering_lib':'covering'}).reset_index(drop=True)

mapping_star_all.loc[lambda df : df.mapped != True,['mapped']] = False
mapping_star_all.loc[lambda df : (df.mapped == False)&(df.covering == False),['classe']] = 'TN'
mapping_star_all.loc[lambda df : (df.mapped == False)&(df.covering == True),['classe']] = 'FN'

mapping_star_mixed = mapping_star_mixed.merge(
            library,how='outer',
            left_on='read',
            right_index=True,
            suffixes=('','_lib')
        ).loc[
            :,
            [c if c != 'covering' else 'covering_lib' for c in mapping_star_mixed.columns.to_list()]
        ].rename(columns={'covering_lib':'covering'}).reset_index(drop=True)

mapping_star_mixed.loc[lambda df : df.mapped != True,['mapped']] = False
mapping_star_mixed.loc[lambda df : (df.mapped == False)&(df.covering == False),['classe']] = 'TN'
mapping_star_mixed.loc[lambda df : (df.mapped == False)&(df.covering == True),['classe']] = 'FN'


<a id='desc_frs'></a>
## Description of FRS (Full-Random Simulation) dataset

<a id='desc_pa'></a>
### Description of the three FRS pseudo-assemblies 

#### FRS pseudo-assembly without introns: FRS_align-validation_wo-introns.fa

   This is the set of full-random simulated contigs without introns. It contains 2000 contigs which have a length beetween 250b to 1500b. 

In [78]:
#print("Assemblathon Analysis of FRS_align-validation_wo-introns.fa :")
assemblathon_wo = parse_assemblathon("./FRS_align_validation_wo-introns.assemblathon.txt","FRS without introns")

#### FRS pseudo-assembly where introns were inserted in each contig : FRS_align-validation_all-wi-introns.fa

   This is the same set of simulated contigs of FRS_align-validation_wo-introns.fa but where a full-random simulated intron has been inserted in each intron (i.e. each contig contains one full-random intron). Inserted introns have a length beetween 250b and 750b.

In [79]:
#print("Assemblathon Analysis of FRS_align-validation_all-wi-introns.fa :")
assemblathon_all = parse_assemblathon("./FRS_align_validation_all-wi-introns.assemblathon.txt", "FRS all modified")

#### FRS pseudo-assembly where introns were inserted in a random half of the contigs : FRS_align-validation_half-wi-introns.fa

   To generate this pseudo-assembly, the two previous pseudo-assembly have been concatenated (i.e. FRS_align-validation_half-wi-introns.fa = FRS_align-validation_wo-introns.fa + FRS_align-validation_all-wi-introns.fa). Therefore, the contigs are, in this dataset, in two states : one version without intron and another version with an introns with a length beetween 250b and 750b. 

In [80]:
#print("Assemblathon Analysis of FRS_align-validation_half-wi-introns.fa :")
assemblathon_half = parse_assemblathon("./FRS_align_validation_half-wi-introns.assemblathon.txt","FRS Mixed states")

<a id='table_1'></a>
**Table 1** *Comparison  of the three pseudo-assembly with Assemblathon.pl statistics*

In [81]:
display(pd.concat([assemblathon_wo,assemblathon_all,assemblathon_half],axis=1))

,FRS without introns,FRS all modified,FRS Mixed states
Number of contigs,2000,2000,4000
Total size of contigs,1759917,2755522,4515439
Longest contig,1499,2231,2231
Shortest contig,250,508,250
Number of contigs > 1K nt,815 40.8%,1596 79.8%,2411 60.3%
N50 contig length,1081,1522,1320
L50 contig count,687,777,1399


<a id='fig_1'></a>
**Fig 1** *Checking of intron insertion in the pseudo-assemblies*

We check that the intron insertion was performed correctly and is random and uniform along the contigs.

In [82]:
def plot_insertion_in_contig(positions) :
    hist = go.Histogram(
            x=positions,
            xbins=dict(
                start=0,
                end=100,
                size=2),
            marker=dict(
                color='purple'
            )
    )
    layout = go.Layout(title='Distribution of introns insertion position along the contigs',
                       xaxis=dict(
                           title="% of contig length"),
                       yaxis=dict(
                           title="Count"))
    fig = go.Figure(data=[hist],layout=layout)
    py.offline.iplot(fig, filename='distrib_intron_insertion')
    
plot_insertion_in_contig(introns['pos_on_contig'])

<a id='desc_lib'></a>
### Description of the FRS reads library

   The reads library has been simulated from the original pseudo-assembly WITHOUT introns with grinder software. It's a paried-end library, with a coverage of 100X. The abundance model follows a uniform law (i.e. all contigs are almost identically covered). The read length folows a normal distribution with N(100;7.5) for parameters. A mutation apparition model is used ; it corresponds to the mutation model of Illumina sequencing : a polynomial (degree 4) law  wich is 3e-3 + 3.3e-8 * i⁴. 80% of these mutations are subsitutions and 20% are indels (Sanger proportions). 
    Finally, two files were generated : FRS_aln-vld_wo-introns_library_read_1.fastq.gz and FRS_aln-vld_wo-introns_library_read_2.fastq.gz

In [83]:
print("Size of the entire library : "+str(len(library))+" reads.")

Size of the entire library : 2000000 reads.


<a id='fig_2'></a>
**Fig 2** *Checking of abundance model of the library*

In [84]:
def parse_rank_file(rank_file) :
    with open(rank_file,"r") as rf :
        ranks = [ligne.lstrip("# ").split("\t") for ligne in rf.read().rstrip().split("\n")]
    return pd.DataFrame(data = ranks[1:], columns=ranks[0] )

ranks = parse_rank_file("FRS_aln-vld_wo-introns_library-ranks.txt")
real = pd.DataFrame((library.groupby('contig').size()/len(library))*100,columns = ['real_abund_perc']).reset_index()
abund = ranks.merge(real,right_on = 'contig',left_on='seq_id',suffixes = ('_grinder','_real'))

fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x = abund['rank'],
        y = abund['real_abund_perc'],
        mode = 'lines',
        name = 'Simulated abundance model'
    )
)

fig.add_trace(
    go.Scatter(
        x = abund['rank'],
        y = abund['rel_abund_perc'],
        mode = 'lines',
        name ='Waited abundance model'
    )
)

fig.update_layout(
    title='Percentage of abundance of each contig in the Full Random generated library',
    xaxis=dict(title="Contigs"),
    yaxis=dict(title="Relative Abundance percentage",
              range=[-0.25,0.5])
)

fig.show()

<a id='desc_bam'></a>
### Description of the four pseudo-assembly/library alignments performed with Hisat2 and STAR

   Each pseudo-assembly WITH introns have been aligned with the library with each of the two aligners. Therefore, we have four alignments :
    
   *  FRS_align-validation_all-wi-introns.fa / FRS-library with Hisat2 : FRS_aln-vld_all-wi-introns_align-hisat.Aligned.sortedByCoord.out.bam
   *  FRS_align-validation_half-wi-introns.fa / FRS-library with Hisat2 : FRS_aln-vld_half-wi-introns_align-hisat.Aligned.sortedByCoord.out.bam
   *  FRS_align-validation_all-wi-introns.fa / FRS-library with STAR : FRS_aln-vld_all-wi-introns_align-star.Aligned.sortedByCoord.out.bam
   *  FRS_align-validation_half-wi-introns.fa / FRS-library with STAR : FRS_aln-vld_half-wi-introns_align-star.Aligned.sortedByCoord.out.bam
    
   All the alignments have been performed with defaults parameters of corresponding aligner.
        

In [85]:


def parse_flagstat(filename : str , lib_size : int, name : str) :
    flagstat = OrderedDict({})
    with open(filename,"r") as f :
        
        first = f.readline().split(" ",3)
        if int(first[2]) == 0 :
            qc_failed = False
            flagstat["Total count"] = int(first[0])
        else :
            qc_failed = True
            flagstat["Total count"] = [int(first[0]),int(first[2])]
            
        items_of_interest=["secondary",
                           "supplementary",
                           "duplicates",
                           "mapped",
                           "properly paired",
                           "singletons",
                           "with mate mapped to a different chr"]
        
        for ligne in f :
            values = ligne.rstrip().split(" ",3)
            if not qc_failed and not int(values[0]) == 0 :
                item = values[-1].split(" (")[0]
                if item in items_of_interest :
                    if item in ["mapped","properly paired","singletons"] :
                        flagstat[item] = "{value} ({percentage}%)".format(
                            value=values[0],
                            percentage=round((int(values[0])/(lib_size+flagstat["secondary"]))*100,2))
                    else :
                        flagstat[item] = int(values[0])
                    
            elif qc_failed and (not int(values[0]) == 0 or not int(values[2]) == 0) :
                item = values[-1].split(" (")[0]
                if item in items_of_interest :
                    if item in ["mapped","properly paired","singletons"] :
                        flagstat[item] = [
                            "{value} ({percentage}%)".format(
                                value=values[0],
                                percentage=round((int(values[0])/(lib_size+flagstat["secondary"]))*100,2)
                            ),
                            "{value} ({percentage}%)".format(
                                value=values[2],
                                percentage=round((int(values[2])/(lib_size+flagstat["secondary"]))*100,2)
                            )]
                    else :
                        flagstat[item] = [int(values[0]),int(values[2])]
            if item == "with mate mapped to a different chr" :
                break
        if not qc_failed :
            return pd.DataFrame.from_dict(flagstat,"index",columns=[name])
        else :
            return pd.DataFrame.from_dict(flagstat,"index",columns=pd.MultiIndex.from_tuples([(name,"QC-passed"),(name,"QC-failed")]))


In [86]:

flag_all_hisat = parse_flagstat("./FRS_aln-vld_all-wi-introns_align-hisat.Aligned.sortedByCoord.out.flagstat.txt", len(library),"All with introns - Hisat2")
flag_half_hisat = parse_flagstat("./FRS_aln-vld_half-wi-introns_align-hisat.Aligned.sortedByCoord.out.flagstat.txt", len(library),"Mix-states contigs - Hisat2")
flag_all_star = parse_flagstat("./FRS_aln-vld_all-wi-introns_align-star.Aligned.sortedByCoord.out.flagstat.txt", len(library),"All with introns - STAR")
flag_half_star = parse_flagstat("./FRS_aln-vld_half-wi-introns_align-star.Aligned.sortedByCoord.out.flagstat.txt", len(library),"Mix-states contigs - STAR")

<a id='table_2'></a>
**Table 2** *Comparison  of the four alignements with samtools flagstats statistics*


In [87]:
print("Total counts of reads to map (library size) : "+str(len(library))+" reads\n")
display(pd.concat([flag_all_hisat,flag_half_hisat,flag_all_star,flag_half_star],axis=1,sort=False).fillna(0))

Total counts of reads to map (library size) : 2000000 reads



,All with introns - Hisat2,Mix-states contigs - Hisat2,All with introns - STAR,Mix-states contigs - STAR
Total count,2000093,3537028,2005764,3598007
secondary,93,1537028,5764,1598007
mapped,1996419 (99.82%),3533998 (99.91%),1900365 (94.75%),3492420 (97.07%)
properly paired,1883386 (94.16%),1884316 (53.27%),1891602 (94.31%),1891402 (52.57%)
singletons,3242 (0.16%),3008 (0.09%),2999 (0.15%),3011 (0.08%)
with mate mapped to a different chr,6,53122,0,0


<a id='fig_3'></a>
**Fig 3** *Dotplots of samtools idxstats results (contigs alignement covering) beetween the alignements of Hisat2 and Star for each reference pseudo-assembly.*

In [151]:
def parse_idxstats(idx) :
    return pd.read_csv(
        idx,
        sep='\t',
        header=None,
        names = ['contig','length','mapped_reads','unmapped_reads'],
        index_col = 'contig'
    )

hisat_all_idxstats = parse_idxstats('FRS_aln-vld_all-wi-introns_align-hisat.Aligned.sortedByCoord.out.idxstats.txt')
star_all_idxstats = parse_idxstats('FRS_aln-vld_all-wi-introns_align-star.Aligned.sortedByCoord.out.idxstats.txt')

all_idxstats = hisat_all_idxstats.join(star_all_idxstats,lsuffix='_Hisat',rsuffix='_Star').sort_values(by='mapped_reads_Hisat')

fig  = go.Figure()

fig.add_trace(
    go.Scatter(
        x = all_idxstats['mapped_reads_Hisat'],
        y = all_idxstats['mapped_reads_Star'],
        mode='markers',
        name='Contigs',
        text=all_idxstats.index
    )
)

fig.add_trace(
    go.Scatter(
        x=list(range(0,2000)),
        y=list(range(0,2000)),
        mode='lines',
        name='Diagonal'
    )
)

fig.update_layout(
    title = 'Dotplot of read quantification by contig according to aligners (All with introns)',
    width=700,
    height=700,
    xaxis=dict(title='Hisat idxstats',range=[0,2000]),
    yaxis=dict(title='Star idxstats',range=[0,2000])
)

fig.show()

In [152]:
hisat_mix_idxstats = parse_idxstats('FRS_aln-vld_half-wi-introns_align-hisat.Aligned.sortedByCoord.out.idxstats.txt')
star_mix_idxstats = parse_idxstats('FRS_aln-vld_half-wi-introns_align-star.Aligned.sortedByCoord.out.idxstats.txt')

half_idxstats = hisat_mix_idxstats.join(star_mix_idxstats,lsuffix='_Hisat',rsuffix='_Star').sort_values(by='mapped_reads_Hisat')


fig  = go.Figure()

fig.add_trace(
    go.Scatter(
        x = half_idxstats.loc[lambda df : df.index.str.endswith('.ori')==True,'mapped_reads_Hisat'],
        y = half_idxstats.loc[lambda df : df.index.str.endswith('.ori')==True,'mapped_reads_Star'],
        mode='markers',
        name='Contigs without intron',
        marker_color='orchid',
        text=half_idxstats.loc[lambda df : df.index.str.endswith('.ori')==True].index
    )
)

fig.add_trace(
    go.Scatter(
        x = half_idxstats.loc[lambda df : df.index.str.endswith('.ori')==False,'mapped_reads_Hisat'],
        y = half_idxstats.loc[lambda df : df.index.str.endswith('.ori')==False,'mapped_reads_Star'],
        mode='markers',
        name='Contigs with intron(s)',
        marker_color='teal',
        text=half_idxstats.loc[lambda df : df.index.str.endswith('.ori')==False].index
    )
)

fig.add_trace(
    go.Scatter(
        x=list(range(0,2000)),
        y=list(range(0,2000)),
        mode='lines',
        name='Diagonal',
        marker_color = 'red'
    )
)

fig.update_layout(
    title = 'Dotplot of read quantification by contig according to aligners (Mixed-states)',
    width=700,
    height=700,
    xaxis=dict(title='Hisat idxstats',range=[0,2000]),
    yaxis=dict(title='Star idxstats',range=[0,2000])
)

fig.show()

<a id='ana_frs'></a>
## Analysis of alignments by seaching split reads  and comparing with simulated introns

<a id='ana_signal'></a>
### Split read signal analysis

<a id='fig_4'></a>
**Fig 4** *Effectives table and barplots of split reads signal detection for STAR and HiSAT2 for the two types of reference.*

In [88]:
def plot_class_reads(*args,**kwargs) :
    series = [[],[]]
    colors = kwargs['colors']
    
    fig = psp.make_subplots(
        rows = 2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        specs=[[{"type":"table"}],
               [{"type":"bar"}]])

    for name,val in args :
        for idx,val in enumerate([val,val.loc[lambda df : df.contig == df.contig.str.rstrip(".ori"),:]]) :
            s = val.loc[:,'classe'].value_counts()
            s.name=name
            
        
            if idx == 0 :
                visibility = True
            else :
                visibility = False
        
            fig.add_trace(
                go.Bar(
                    visible=visibility,
                    x=s.index[1:],
                    y=s.values[1:]/s.sum()*100,
                    name = s.name,
                    marker_color=colors[s.name]
                ),row=2,col=1)
            
            s['Total'] = s.sum()
            series[idx].append(s)

    for idx,serie in enumerate(series) :
        table = pd.concat(serie,axis=1,sort=False).fillna(0)
        
        
        if idx == 0 :
            visibility = True
        else :
            visibility = False
            
        fig.add_trace(
            go.Table(
                visible=visibility,
                columnwidth=[40,80],
                header = dict(
                    values=["",*[c.join(["<b>","</b>"]) for c in table.columns.to_list()]],
                    fill_color='lightgrey'
                    ),
                cells = dict(
                    values=[[v.join(["<b>","</b>"]) for v in list(table.reset_index().T.values)[0]],
                           *list(table.reset_index().T.values)[1:]],
                    fill=dict(color=['lightgrey','lavender'])
                    )
                ),
            row=1,col=1)
    
#     print(vars(fig.layout))
    
    fig.update_layout(
        height=700,
        legend=dict(
            x=-0.4,y=0.40
            ),
        updatemenus=[
            go.layout.Updatemenu(
                buttons=[
                    dict(
                        args=[{"visible":[True,False]}],
                        label='All alignements',
                        method='restyle'
                        ),
                    dict(
                        args=[{"visible":[False,True]}],
                        label='Alignements on contigs with introns',
                        method='restyle'
                        ),
                    ]
                )
            ]
        )
    fig.show()
    
plot_class_reads(*zip(names,[
                    mapping_hisat_all,
                    mapping_hisat_mixed,
                    mapping_star_all,
                    mapping_star_mixed
                    ]),
                colors=colors)

<a id='fig_5'></a>
**Fig 5** *Counting table and barplots of mapped covering reads' main characteristics.*

In [89]:
def plot_covering_reads(*args,**kwargs) :
    series=[]
    fig = go.Figure()
    for name, val in args :
        s = pd.Series(name=name)
        s['Covering']=len(val)
        s['Unmapped']=len(val.loc[lambda df : df.mapped == False])
        tmp = val.merge(library,left_on='read',right_index=True,suffixes=("","_lib"))
        s['Mismapped'] = len(tmp.loc[lambda df : (df.contig.str.rstrip('.ori') != df.contig_lib.str.rstrip('.ori'))& (df.mapped == True)])
        s['Unsplit'] = len(val.loc[lambda df : (df.mapped==True)&(df.split==False)])
        s['Missplit'] = len(val.loc[lambda df : (df.split==True)&(df.missplit==True)])
        s['Correct splitting'] = len(val.loc[lambda df : df.classe == 'TP'])
        series.append(s)
        
        to_plot = s[['Unmapped','Unsplit','Correct splitting']]/s['Covering']*100
        fig.add_trace(
            go.Bar(
                    x=to_plot.index,
                    y=to_plot.values,
                    name = s.name,
                    marker_color=colors[s.name]
            ))
    table = pd.concat(series,axis=1,sort=False)
    display(table)
    
    fig.update_layout(
        title='Global mapping results on introns-covering reads',
        xaxis=dict(title='Lectures charecteristics'),
        yaxis=dict(title='Percentage of total covering reads alignements')
        )
    
    fig.show()

plot_covering_reads(*zip(names,[
                    mapping_hisat_all.loc[lambda df : df.covering == True,:],
                    mapping_hisat_mixed.loc[lambda df : df.covering == True,:],
                    mapping_star_all.loc[lambda df : df.covering == True,:],
                    mapping_star_mixed.loc[lambda df : df.covering == True,:]
                    ]),
                colors=colors,
                library=library)

,All with introns - Hisat2,Mix-states contigs - Hisat2,All with introns - STAR,Mix-states contigs - STAR
Covering,226831,227758,227442,287314
Unmapped,730,215,10801,10993
Mismapped,1,0,0,0
Unsplit,7803,49064,33746,221723
Missplit,66,4,260,29
Correct splitting,218232,178475,182635,54545


<a id='fig_6'></a>
**Fig 6** *Counting table and barplots of aligners' duplication capacity of covering reads in mix-states reference.*

In [161]:
def plot_reads_duplication(*args,**kwargs) :
    series=[]
    fig = go.Figure()
    for name,val in args :
        s = pd.Series(name=name)
        mer_val = val.merge(library,left_on='read',right_index=True,suffixes=("","_lib")).loc[lambda df : (df.contig.str.rstrip('.ori') == df.contig_lib.str.rstrip('.ori'))]
        s['Correctly mapped covering reads'] = len(mer_val)
        no_dupl= mer_val.loc[mer_val['read'].duplicated(keep=False)==False,:]
        s['Mapped only on full contig'] =len(no_dupl.loc[lambda df : df.contig.str.endswith('.ori') == True])
        s['Mapped only on split contig'] =len(no_dupl.loc[lambda df : df.contig.str.endswith('.ori') == False])
        s['Unsplit/Missplit on split contig']=len(no_dupl.loc[lambda df : (df.contig.str.endswith('.ori') == False)&((df.split==False)|(df.missplit==True))])
        s['Correct only on split contig']=len(no_dupl.loc[lambda df : (df.contig.str.endswith('.ori') == False)&(df.split==True)&(df.missplit==False)])
        dupl = mer_val.loc[mer_val['read'].duplicated(keep=False)==True,:]
        s['Duplicated on both contigs']= len(dupl)
        s['Duplicated and missplit/unsplit on split contig']=len(dupl.loc[lambda df : (df.contig.str.endswith('.ori') == False)&((df.split==False)|(df.missplit==True))])
        s['Duplicated and correct split']=len(dupl.loc[lambda df : (df.contig.str.endswith('.ori') == False)&(df.split==True)&(df.missplit==False)])
        series.append(s)
        
        fig.add_trace(
            go.Bar(
                x=s.index,
                y=s.values,
                name = s.name,
                marker_color=colors[s.name],
                text=s.values,
                textposition='auto'    
            ))
    table = pd.concat(series,axis=1,sort=False)
    display(table)
    
    fig.update_layout(
        title='Analysis of duplicated alignements on the two-satest contigs (all introns-covering reads)',
        xaxis=dict(title='Groups'),
        yaxis=dict(title='Count')
        )
    
    fig.show()

plot_reads_duplication(*zip([names[1],names[3]],[
                    mapping_hisat_mixed.loc[lambda df : df.covering == True,:],
                    mapping_star_mixed.loc[lambda df : df.covering == True,:]
                    ]),
                colors=colors,
                library=library)

,Mix-states contigs - Hisat2,Mix-states contigs - STAR
Correctly mapped covering reads,227754,276321
Mapped only on full contig,47017,155296
Mapped only on split contig,178801,281
Unsplit/Missplit on split contig,331,257
Correct only on split contig,178470,24
Duplicated on both contigs,1936,120744
Duplicated and missplit/unsplit on split contig,964,5345
Duplicated and correct split,5,54521


In [91]:
# def print_global_results(global_dfs : list,complement="") :
#     print("Table of global intron detection results "+complement)
#     display(pd.concat(global_dfs,axis = 1))



# def plot_percent_correct(series : list,complement="") :
#     boxplots=[]
#     for s in series :
#         box = go.Box(
#             x=s.values,
#             name=s.name)
#         boxplots.append(box)
    
#     layout = go.Layout(title= "Boxplot of correct lectures percentage for VP class introns "+complement,
#             xaxis = dict(title="%_correct")) 
#     fig = go.Figure(data=boxplots,layout=layout)
#     py.offline.iplot(fig,filename="box_%_correct")
    
    


# def plot_introns_detection(series : list,complement="") :
#     barplots=[]
#     print("Table of intron detection")
#     display(pd.concat(series,axis=1))
#     for s in series :
#         bar = go.Bar(
#             x=s.index,
#             y=s.values,
#             name = s.name,
#             text=s.values,
#             textposition="auto"
#             )
#         barplots.append(bar)
    
#     layout = go.Layout(title='Global results on introns detection '+complement,
#                        xaxis=dict(
#                            title="Groups"),
#                        yaxis=dict(
#                            title="Count"))
#     fig = go.Figure(data=barplots,layout=layout)
#     py.offline.iplot(fig, filename='introns_detection_bar')



# def norm_each_bin(bining) :
    
#     if bining["correct"].all() and not bining.empty  :
#         return float(100)
#     elif bining["correct"].any() and not bining.empty :
#         return (bining.groupby("correct").size()[True]/len(bining))*100
#     else :
#         return float(0)
    
# def histplot_pos_on_reads(data : list) :
#     hists=[]
#     for d in data :
#         df = list(d.values()).pop()
#         name = list(d.keys()).pop()
#         factor = pd.cut(list(df["position"]), range(0,101,2),right=False)
#         positions_norm = df.groupby(factor).apply(norm_each_bin)
#         hist = go.Bar(
#             x=[n+1 for n in range(0,100,2)],
#             y=positions_norm.values,
#             width=[2]*50,
#             name = name,
#             hovertemplate="<b>Value : %{y:.2f}</b><br>"
#                           "<i>Interval : %{text}</i>",
#             text = ["{m}-{n}".format(m=k,n=k+2) for k in range(0,100,2)]
#             )
#         hists.append(hist)
#     layout = go.Layout(
#         title='Correct spliting counting depending on intron insertion position on read (in % of read length)',
#         xaxis1=dict(
#             title='% of read length'
#         ),
#         xaxis2=dict(
#             title='% of read length'
#         ),
#         xaxis3=dict(
#             title='% of read length'
#         ),
#         xaxis4=dict(
#             title='% of read length'
#         ),
#         yaxis1=dict(
#             title='% of correct split reads'
#         ),
#         yaxis2=dict(
#             title='% of correct split reads'
#         ),
#         yaxis3=dict(
#             title='% of correct split reads'
#         ),
#         yaxis4=dict(
#             title='% of correct split reads'
#         )
#     )
#     fig = tools.make_subplots(rows=2, cols=2,shared_yaxes=True)
#     fig.append_trace(hists[0], 1, 1)
#     fig.append_trace(hists[1], 1, 2)
#     fig.append_trace(hists[2], 2, 1)
#     fig.append_trace(hists[3], 2, 2)
#     fig['layout'].update(layout)
#     py.offline.iplot(fig, filename='styled histogram')

# def histplot_pos_on_contig(data : list) :
#     hists=[]
#     for d in data :
#         df = list(d.values()).pop()
#         name = list(d.keys()).pop()
#         factor = pd.cut(list(df["position"]), range(0,101,2),right=False)
#         positions_norm = df.groupby(factor).apply(norm_each_bin)
#         hist = go.Bar(
#             x=[n+1 for n in range(0,100,2)],
#             y=positions_norm.values,
#             width=[2]*50,
#             name = name,
#             hovertemplate="<b>Value : %{y:.2f}</b><br>"
#                           "<i>Interval : %{text}</i>",
#             text = ["{m}-{n}".format(m=k,n=k+2) for k in range(0,100,2)]
#             )
#         hists.append(hist)
#         layout = go.Layout(
#     title='Correct spliting counting depending on intron insertion position on contig (in % of contig length)',
#         xaxis1=dict(
#             title='% of contig length'
#         ),
#         xaxis2=dict(
#             title='% of contig length'
#         ),
#         xaxis3=dict(
#             title='% of contig length'
#         ),
#         xaxis4=dict(
#             title='% of contig length'
#         ),
#         yaxis1=dict(
#             title='% of correct split reads'
#         ),
#         yaxis2=dict(
#             title='% of correct split reads'
#         ),
#         yaxis3=dict(
#             title='% of correct split reads'
#         ),
#         yaxis4=dict(
#             title='% of correct split reads'
#         )
#     )
#     fig = tools.make_subplots(rows=2, cols=2,shared_yaxes=True)
#     fig.append_trace(hists[0], 1, 1)
#     fig.append_trace(hists[1], 1, 2)
#     fig.append_trace(hists[2], 2, 1)
#     fig.append_trace(hists[3], 2, 2)
#     fig['layout'].update(layout)
#     py.offline.iplot(fig, filename='styled histogram')

    

#### Analysis of the four alignements considering all introns-covering reads

In [92]:

# plot_reads_duplication([res[4] for res in analysis_res if res[4] is not None],"(all introns-covering reads)")
# print_global_results([res[0] for res in analysis_res],"(all introns-covering reads)")
# plot_percent_correct([res[2] for res in analysis_res],"(all introns-covering reads)")
# plot_introns_detection([res[3] for res in analysis_res],"(all introns-covering reads)")

In [93]:

# print("Correct splitting depends on intron insertion in READS")
# histplot_pos_on_reads([res[5] for res in analysis_res])
# print("Correct splitting depends on intron insertion in CONTIGS")
# histplot_pos_on_contig([res[6] for res in analysis_res])

#### Analysis of the four alignements considering only the reads where introns are inserted at 10 bases in minimum from borders

In [94]:
# analysis_res_10 = []
# analysis_res_10.append(alignement_analysis("./FRS_aln-vld_all-wi-introns_align-hisat.Aligned.sortedByCoord.out.bam","./FRS_align-validation_all-wi-introns.fa","./FRS_align-validation_all-introns-coord.txt",library,"All with introns - Hisat2 - margin 10",10))
# analysis_res_10.append(alignement_analysis("./FRS_aln-vld_half-wi-introns_align-hisat.Aligned.sortedByCoord.out.bam","./FRS_align-validation_half-wi-introns.fa","./FRS_align-validation_half-introns-coord.txt",library,"Mix-states contigs - Hisat2 - margin 10",10,mix=True))
# analysis_res_10.append(alignement_analysis("./FRS_aln-vld_all-wi-introns_align-star.Aligned.sortedByCoord.out.bam","./FRS_align-validation_all-wi-introns.fa","./FRS_align-validation_all-introns-coord.txt",library,"All with introns - STAR - margin 10",10))
# analysis_res_10.append(alignement_analysis("./FRS_aln-vld_half-wi-introns_align-star.Aligned.sortedByCoord.out.bam","./FRS_align-validation_half-wi-introns.fa","./FRS_align-validation_half-introns-coord.txt",library,"Mix-states contigs - STAR - margin 10",10,mix=True))

# plot_covering_reads([res[1] for res in analysis_res_10],"(only reads with 10 bases margin)")
# plot_reads_duplication([res[4] for res in analysis_res_10 if res[4] is not None],"(only reads with 10 bases margin)")
# print_global_results([res[0] for res in analysis_res_10],"(only reads with 10 bases margin)")
# plot_percent_correct([res[2] for res in analysis_res_10],"(only reads with 10 bases margin)")
# plot_introns_detection([res[3] for res in analysis_res_10],"(only reads with 10 bases margin)")


#### Analysis of the four alignements considering only the reads where introns are inserted at 20 bases in minimum from borders

In [95]:
# analysis_res_20 = []
# analysis_res_20.append(alignement_analysis("./FRS_aln-vld_all-wi-introns_align-hisat.Aligned.sortedByCoord.out.bam","./FRS_align-validation_all-wi-introns.fa","./FRS_align-validation_all-introns-coord.txt",library,"All with introns - Hisat2 - margin 20",20))
# analysis_res_20.append(alignement_analysis("./FRS_aln-vld_half-wi-introns_align-hisat.Aligned.sortedByCoord.out.bam","./FRS_align-validation_half-wi-introns.fa","./FRS_align-validation_half-introns-coord.txt",library,"Mix-states contigs - Hisat2 - margin 20",20,mix=True))
# analysis_res_20.append(alignement_analysis("./FRS_aln-vld_all-wi-introns_align-star.Aligned.sortedByCoord.out.bam","./FRS_align-validation_all-wi-introns.fa","./FRS_align-validation_all-introns-coord.txt",library,"All with introns - STAR - margin 20",20))
# analysis_res_20.append(alignement_analysis("./FRS_aln-vld_half-wi-introns_align-star.Aligned.sortedByCoord.out.bam","./FRS_align-validation_half-wi-introns.fa","./FRS_align-validation_half-introns-coord.txt",library,"Mix-states contigs - STAR - margin 20",20,mix=True))

# plot_covering_reads([res[1] for res in analysis_res_20],"(only reads with 20 bases margin)")  
# plot_reads_duplication([res[4] for res in analysis_res_20 if res[4] is not None],"(only reads with 20 bases margin)")
# print_global_results([res[0] for res in analysis_res_20],"(only reads with 20 bases margin)")
# plot_percent_correct([res[2] for res in analysis_res_20],"(only reads with 20 bases margin)")
# plot_introns_detection([res[3] for res in analysis_res_20],"(only reads with 20 bases margin)")